# C(R)UD: toevoegen (insert), veranderen (update) en verwijderen (delete)

In het vorige notebook hebben aandacht besteed aan het opvragen van gegevens uit de datebase.
In dit notebook gaan we in op het toevoegen (insert of create), veranderen (update) en verwijderen (delete) van documenten in de database. We hebben dan de basisopdrachten (CRUD) van een database-systeem behandeld.

## Initialisaties

In [ ]:
import os
import re
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
import pymongo
from bson.objectid import ObjectId

pd.set_option('max_colwidth',160)

userline = !echo $USER
username = userline[0]
dbname = username + "-demodb"
print("Database name: " + dbname)

print('Mongo version', pymongo.__version__)
client = pymongo.MongoClient('localhost', 27017)
db = client[dbname]
collection = db.contacts

mongopathfile = !cat mongopath
mongopath = mongopathfile[0]

collection.drop()
os.system(mongopath + 'mongoimport -d ' + dbname + ' -c contacts adressen.json')

## Insert: toevoegen van documenten

Met de `insert`-opdracht kunnen we documenten aan een collectie toevoegen.
Vaak gebruiken we `insert_one`, om een enkel document toe te voegen.

**Let op**: als je deze opdracht herhaalt, voeg je nog  nog een kopie van hetzelfde document aan de collectie toe.

**Opdracht** Ga dit na.

In [ ]:
person = {"name": "Sylvia Hansma", 
          "email": "sylh123@hotmail.com", 
          "address": {"street": "Rijksstraatweg 84", "city": "Halfweg"}
         }
collection.insert_one(person)

In [ ]:
list(collection.find({"name": "Sylvia Hansma"}))

### Dubbele inserts

**Let op**: als je deze insert-opdracht herhaalt, voeg je nog  nog een kopie van hetzelfde document aan de collectie toe. (We zullen later behandelen hoe je documenten verwijdert.)

**Opdracht** Ga dit na.

## Update

Door middel van een update-opdracht kun je bepaalde velden veranderen of toevoegen (via `$set`, met een aantal nieuwe waarden).

**Veranderen** van gegevens in bestaande velden (properties):

In [ ]:
upd_obj = {"address.street": "Mozartplein 73", "address.city": "Rotterdam", "address.postcode": "3021 BA"}

collection.update_one({"name": "Anna Verschuur"}, {"$set": upd_obj})
obj = collection.find_one({"name": "Anna Verschuur"})
print(obj)

**Opdracht** Maak een opdracht voor het aanpassen van het email-adres van Hans de Boer in de database.

**Toevoegen** van velden: als een veld in het `$set`-document niet bestaat, wordt dit toegevoegd. (Vgl. ook de *Upsert*, verderop.)

> Dit is in een relationele database alleen mogelijk als je de tabel verandert: je voegt dat één of meer velden voor alle rijen in de tabel toe. In MongoDB kun je dit *per document* doen.

In [ ]:
collection.update_one({"name": "Anna Verschuur"}, {"$set": {"isFamily": True, "email": "anna33@gmail.com"}})

obj = collection.find_one({"name": "Anna Verschuur"})
print(obj)

**Opdracht** Ga na dat velden op deze manier niet tweemaal toegevoegd worden.

### Unset: verwijderen van velden (properties)

Je kunt een ongewenste veld (property) van een document verwijderen met de `$unset`-opdracht. De waarde van de property is in dit geval niet relevant.

> Deze `$unset` kan handig zijn als een foute update resulteert in het toevoegen van een nieuw veld in plaats van in een update van een bestaand veld.

In [ ]:
upd_obj = {"isFamily": True}
collection.update_one({"name": "Anna Verschuur"}, {"$unset": upd_obj})

obj = collection.find_one({"name": "Anna Verschuur"})
print(obj)

## Upsert: overschrijven of toevoegen

Als je een document wilt toevoegen, waarbij je een eventueel bestaand document wilt overschrijven, dan kun je een update doen met `upsert=True`.
Dit betekent dat je een bestaand document overschrijft (update), of, als dat er niet is, een nieuw document toevoegt (insert).

> Hier moet je wel voorzichtig mee zijn: er kunnen bijvoorbeeld meerdere personen met dezelfde naam zijn. 
  Je kunt het query-deel van bijvoorbeeld uitbreiden met het email-adres.

In [ ]:
person = {"name": "Sylvia Hansma", 
          "email": "sylh123@hotmail.com", 
          "address": {"street": "Rijksstraatweg 82", "city": "Halfweg"}
         }
collection.update_one({"name": person["name"]}, {"$set": person}, upsert=True)
list(collection.find())

## Meervoudige waarden (TODO)

**P.M. nog toevoegen: het veranderen van gegevens in het geval van meervoudige waarden, bijvoorbeeld email-adressen**

## Verwijderen van een document (delete)

Voor het verwijderen van documenten heb je twee soorten opdrachten:

* ``delete_one(filter-doc)``
* ``delete_many(filter-doc)``

De eerste opdracht verwijderd ten hoogste 1 document, ook als er meerdere documenten zijn die matchen met het filter-document.

Als er meerdere documenten zijn met dezelfde naam en andere eigenschappen, bijvoorbeeld doordat er per ongeluk een tweede `insert` uitgevoerd is, dan kun je het onbedoelde document verwijderen met behulp van de unieke `_id` in het filter

In [ ]:
list(collection.find({"name": re.compile("Joop")}))

In [ ]:
collection.delete_one({"name": "Joop de Zwart"})

list(collection.find({"name": re.compile("Joop")}))

**Opdracht (a)** zorg ervoor dat er twee documenten zijn met dezelfde inhoud voor "Sylvia Hansma"`.

**Opdracht (b)** Laat zien dat er twee (of meer) documenten zijn met dezelfde inhoud voor "Sylvia Hansma"`.

In [ ]:
list(collection.find({"name": "Sylvia Hansma"}))

**Opdracht (c)** Verwijder het onbedoelde document met behulp van de `_id` in het filter

**Opdracht** Formuleer een opdracht voor het verwijderen van alle documenten in `collection`. Laat zien dat de collection leeg is.

In [ ]:
list(collection.find())